## A Retrieval Augmented Generation Pipeline(RAG) using an Open Source Library for Embeddings which will also act as the vector store from [NeuML](https://github.com/neuml/txtai) called txtai coupled with an Open Source LLM from HuggingFace.


### Instal the required packages

In [ ]:
!pip install txtai[pipeline]          # If running this notebook on colab, you will be asked to restart the notebook after running this cell, please do so

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.1/215.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 115.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 33.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8

In [ ]:
!pip install pypdf
!pip install langchain-text-splitters
!pip install --quiet langchain

  Using cached pypdf-4.2.0-py3-none-any.whl (290 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install accelerate bitsandbytes

In [ ]:
!pip install huggingface-hub

In [ ]:
!pip install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 37.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


### Log into your Hugging-Face Account to choose the open source model of your choice

In [ ]:
from huggingface_hub import notebook_login
                                                              # Copy paste the Hugging-Face access keys in the cell below
notebook_login()

### Use Langchain's PyPDF Loader and text splitter to chunk the pdf and Embed the text as Vectors

In [ ]:
import os
from txtai import Embeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


file_path = r'/content/Mavic_Air_2_User_Manual_v1.4_en.pdf'


loader = PyPDFLoader(f"{file_path}")
pages = loader.load()
text = RecursiveCharacterTextSplitter(
chunk_size=1000, chunk_overlap=250).split_documents(pages)

page_contents = []

for element in text:
    page_contents.append(element.page_content)



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
embeddings = Embeddings(hybrid=True, content=True)              # Embedd the text and store them as Vectors in the Embeddings from txtai
embeddings.index(page_contents)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### Create a Function to Search the Vector Store according to Similiarity Search

In [ ]:
def context(question):
  context =  "\n".join(x["text"] for x in embeddings.search(question))              # A function that takes in question and does a similarity search in the Vector Store with the question
  return context

### Load an Open Source LLM availabe from the Hugging Face Hub

In [ ]:

from txtai.pipeline import LLM
                                                # Download any open source model from Hugging Face Hub by replacing the model ID in the Code below.
llm = LLM("TheBloke/Mistral-7B-OpenOrca-AWQ")   # Note you will need a gpu to load the llm

config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def execute(question, text):                                                                                              # Write a prompt restricting the LLM to only answer Relevant questions to the Retrieved Document
                                                                                                                          # The prompt is structured in such a way because the model is based on Mixtral 8x7B
  prompt = f"""<|im_start|>system
  You are a friendly assistant. You answer questions from users.<|im_end|>
  <|im_start|>user
  Answer the following question using only the context below. Only include information specifically discussed.

  question: {question}
  context: {text} <|im_end|>
  <|im_start|>assistant
  """

  return llm(prompt, maxlength=4096, pad_token_id=32000)                                                                # Return the prompt along with the question and context to the LLM


In [ ]:
def rag(question):                                                                    # A main function to call the entire Pipeline
  return execute(question, context(question))


### Run the entire pipeline with questions about the drone

In [ ]:
result = rag("What is the maximum Range of the Drone?")
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The maximum range of the drone is 6 mi (10 km) using the OCUSYNCTM 2.0 long-range transmission technology built into the remote controller.


In [ ]:
result = rag("What is the maximum air time of the Drone?")
print(result)

The maximum air time of the Drone is 34 minutes.


In [ ]:
result = rag("What happens if the drone runs out of battery midflight?")
print(result)

If the drone runs out of battery midflight, the Low Battery RTH feature is triggered. The aircraft will automatically return to the Home Point if no action is taken after a 10-second countdown. If the user cancels RTH, the drone may not have enough power to land safely, potentially causing it to crash or be lost. If the current battery level can only support the aircraft long enough to descend from its current altitude, the drone will land automatically. However, auto landing cannot be canceled. It is recommended to discharge the Intelligent Flight Batteries to 30% or lower before transportation.


In [ ]:
result = rag("What are the indication lights on the drone for?")
print(result)

The indication lights on the drone serve various purposes, such as communicating the status of the aircraft's flight control system. The front LEDs show the orientation of the aircraft, and they glow solid red when the aircraft is turned on to indicate the front of the aircraft. The aircraft status indicators have different colors and blinking patterns to represent different states, such as normal, warning, and flight recorder. These lights help the user understand the drone's status, such as GPS signal strength, battery level, and whether the vision systems are available. Additionally, the lights can indicate safety settings, control, and transmission information.


In [ ]:
result = rag("What is the specifications for the drone's camera?")
print(result)

saved.

Specifications for the drone's camera:
- Fully stabilized 3-axis gimbal
- 1/2" sensor camera
- Shoots 4K/60 fps video and 48 MP photos
- Updated Hyperlapse feature supports 8K timelapse
- Video coding format: H.265
- Max bitrate: 12 Mbps
- App: DJI Fly (iOS v10.0.2 or later; Android v6.0 or later)
- Supported SD cards: UHS-I Speed Grade 3 rating microSD card
- Camera settings: Before use, ensure they are configured as desired
- Camera parameters: Not saved if the aircraft is powered off incorrectly
- Single video recordings are limited to 30 minutes


In [ ]:
result = rag("What is the Default Height Limit for the Drone?")
print(result)

The default height limit for the drone is 16 ft (5 m) when the GPS signal is weak and Downward Vision System is activated. When the GPS signal is weak and Downward Vision System is inactive, the height limit is 98 ft (30 m).


In [ ]:
result = rag("Can the users set custom flight limits for the drone?")
print(result)

Yes, the users can set custom flight limits for the drone. They can change the flight altitude and distance limits in DJI Fly. Based on these settings, the aircraft will fly in a restricted cylinder, with the maximum flight altitude and maximum radius as the limits.


In [ ]:
result = rag("My drone crashed during flight, what can I do to repair the drone?")
print(result)

1. Check if there is nothing obstructing the motors and that they are functioning normally.
2. Ensure that DJI Fly is successfully connected to the aircraft.
3. Make sure that the camera lens and Vision System sensors are clean.
4. Use only genuine DJI parts or parts certified by DJI. Unauthorized parts or parts from non-DJI certified manufacturers may cause system malfunctions and compromise safety.
5. If the drone is still not functioning properly, consider seeking professional help or contacting DJI's customer support for further assistance.
